In [26]:
!pip install --upgrade goatools


# Goatools for Graph Communities  

In [28]:
import os
from collections import defaultdict
from goatools.obo_parser import GODag
from goatools.goea.go_enrichment_ns import GOEnrichmentStudy
import sys

In [29]:
def load_gene2go(file_path):
    """Load the gene-to-GO mapping file into a dictionary."""
    gene2go = defaultdict(set)
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split('\t')
            gene = parts[0]
            go_terms = parts[1].split(';') if len(parts) > 1 else []
            for go_term in go_terms:
                gene2go[gene].add(go_term)
    return dict(gene2go)

In [30]:
def load_population(file_path):
    """Load the population file into a set."""
    with open(file_path, 'r') as file:
        return set(line.strip() for line in file)

In [31]:
def load_communities(file_path):
    """Load the communities file into a dictionary."""
    communities = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(':')
            if len(parts) != 2:
                raise ValueError(f"Invalid format in communities file: {line}")
            community_id = parts[0].strip()
            genes = set(parts[1].split(','))
            communities[community_id] = genes
    return communities

In [32]:
def perform_enrichment_analysis_with_goatools(community_id, community_genes, population, gene2go, go_dag, output_file, significant_genes, pval_threshold=0.05):
    """
    Perform Gene Ontology enrichment analysis for a single community using GOATOOLS.
    """
    # Create the GO enrichment object
    goea_obj = GOEnrichmentStudy(
        population,  # Background population
        gene2go,     # Gene-to-GO mapping
        go_dag,      # Gene Ontology DAG
        methods=['fdr_bh'],  # Multiple test correction
        log=sys.stdout  # Debug output
    )

    # Perform the enrichment analysis
    results = goea_obj.run_study(community_genes)

    # Calculate GO term frequencies in the population
    go_population_count = defaultdict(int)
    for gene in population:
        for go_term in gene2go.get(gene, []):
            go_population_count[go_term] += 1

    # Calculate GO term frequencies in the community
    go_community_count = defaultdict(int)
    for gene in community_genes:
        for go_term in gene2go.get(gene, []):
            go_community_count[go_term] += 1

    # Write results for this community in TSV format
    with open(output_file, 'w') as file:
        # Write the header row
        #print(f"GO Term: {res.GO}, Raw P-value: {res.p_uncorrected:.6f}, FDR: {res.p_fdr_bh:.6f}")  # Debugging output

        file.write("GO_Term\tDescription\tP-value\tOdds_Ratio\tFDR\tSignificant_Genes\tGenes_in_Community\tGenes_in_Population\n")
        
        # Write each result as a row
        for res in results:
            if res.p_fdr_bh <= pval_threshold:
                go_term_genes_in_community = go_community_count[res.GO]
                go_term_genes_in_population = go_population_count[res.GO]

                file.write(f"{res.GO}\t{res.name}\t{res.p_uncorrected:.6f}\t"
                           f"{res.ratio_in_study[0]/res.ratio_in_study[1]:.4f}\t"
                           f"{res.p_fdr_bh:.6f}\t{','.join(res.study_items)}\t"
                           f"{go_term_genes_in_community}\t{go_term_genes_in_population}\n")
                
                # Add significant genes to the set
                significant_genes[community_id].update(res.study_items)

In [33]:
def main_with_goatools(population_file, gene2go_file, communities_file, go_obo_file, individual_output_dir, summary_file, pval_threshold=0.05):
    """Main function to run GO enrichment analysis for all communities."""
    # Load data
    population = load_population(population_file)
    gene2go = load_gene2go(gene2go_file)
    communities = load_communities(communities_file)
    go_dag = GODag(go_obo_file)

    # Create output directory if it doesn't exist
    os.makedirs(individual_output_dir, exist_ok=True)

    # Dictionary to store significant genes for each community
    significant_genes = defaultdict(set)

    # Perform enrichment analysis for each community
    for community_id, community_genes in communities.items():
        print(f"Processing Community {community_id} with {len(community_genes)} genes...")

        matched_genes = [gene for gene in community_genes if gene in gene2go]
        if not matched_genes:
            print(f"Warning: No matching GO terms for community {community_id}. Skipping...")
            continue

        output_file = os.path.join(individual_output_dir, f"Graph_community_{community_id}_enrichment.tsv")

        perform_enrichment_analysis_with_goatools(
            community_id, community_genes, population, gene2go, go_dag, output_file, significant_genes, pval_threshold
        )

    # Write summary file in TSV format
    with open(summary_file, 'w') as file:
        file.write("Community\tSignificant_Genes\n")
        for community_id, genes in significant_genes.items():
            file.write(f"{community_id}\t{','.join(genes)}\n")

In [34]:
if __name__ == "__main__":
    try:
        main_with_goatools(
            population_file="Population.txt",
            gene2go_file="gene2golist2.txt",
            communities_file="graph_communities.txt",
            go_obo_file="go-basic.obo",
            individual_output_dir="1_GOenrichment_results_Graph_Final",
            summary_file="1_G0significant_genes_summary.tsv",
            pval_threshold=0.05
        )
    except Exception as e:
        print(f"An error occurred: {e}")


go-basic.obo: fmt(1.2) rel(2024-10-27) 44,017 Terms
Processing Community Community 0 with 6 genes...

Load  Ontology Enrichment Analysis ...
Propagating term counts up: is_a
100%  1,096 of  1,097 population items found in association

Runing  Ontology Analysis: current study set of 6 IDs.
100%      5 of      5 study items found in association
 83%      5 of      6 study items found in population(1097)
Calculating 3,565 uncorrected p-values using fisher_scipy_stats
   3,565 terms are associated with  1,096 of  1,097 population items
      72 terms are associated with      5 of      6 study items
  METHOD fdr_bh:
      26 GO terms found significant (< 0.05=alpha) ( 26 enriched +   0 purified): statsmodels fdr_bh
       5 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)
Processing Community Community 1 with 7 genes...

Load  Ontology Enrichment Analysis ...
Propagating term counts up: is_a
100%  1,096 of  1,097 po

# HyperGraph 

In [48]:
if __name__ == "__main__":
    try:
        main_with_goatools(
            population_file="Population.txt",
            gene2go_file="gene2golist2.txt",
            communities_file="hypergraph_communities.txt",
            go_obo_file="go-basic.obo",
            individual_output_dir="1_GOenrichment_results_HyperGraph_Final",
            summary_file="1_G0significant_genes_HyperGraph_summary.tsv",
            pval_threshold=0.05
        )
    except Exception as e:
        print(f"An error occurred: {e}")


go-basic.obo: fmt(1.2) rel(2024-10-27) 44,017 Terms
Processing Community Community 0 with 6 genes...

Load  Ontology Enrichment Analysis ...
Propagating term counts up: is_a
100%  1,096 of  1,097 population items found in association

Runing  Ontology Analysis: current study set of 6 IDs.
100%      5 of      5 study items found in association
 83%      5 of      6 study items found in population(1097)
Calculating 3,565 uncorrected p-values using fisher_scipy_stats
   3,565 terms are associated with  1,096 of  1,097 population items
      72 terms are associated with      5 of      6 study items
  METHOD fdr_bh:
      26 GO terms found significant (< 0.05=alpha) ( 26 enriched +   0 purified): statsmodels fdr_bh
       5 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)
Processing Community Community 1 with 7 genes...

Load  Ontology Enrichment Analysis ...
Propagating term counts up: is_a
100%  1,096 of  1,097 po